# Stochastic simulation assignment 2
## Peter Voerman and Nick van Santen
### 11749547 and ...

In [7]:
import math
import numpy as np
import random
import simpy
import bisect

In [8]:

class Server():

    def __init__(self, env, n_machines):
        self.env = env
        self.machine = simpy.PriorityResource(env, n_machines)

    def run_task(self, waiting_time):
        yield self.env.timeout(waiting_time)

    @property
    def queue_length(self):
        return len(self.machine.queue)


def process_task(env, server, data, waiting_times, duration, shortest_job_first, time_choice):
    if time_choice == 'm':
        waiting_time = generate_random_markov(duration)
        if shortest_job_first:
            bisect.insort(waiting_times, waiting_time)
            index = (waiting_times.index(waiting_time))

            for request in server.machine.queue:
                if request.priority >= index:
                    request.priority += 1
        else:
            index = 1
    if time_choice == 'd':
        waiting_time = duration
        index = 1
    

    #print(f"{task} added to server at {env.now}. Queue length: {server.queue_length}")
    time_at_queue = env.now
    with server.machine.request(priority=index) as request:
        yield request

        #print(f"{task} is being processed at {env.now}")
        time_start_process = env.now
        yield env.process(server.run_task(waiting_time))
        #print(f"{task} is completed at {env.now}")
        time_end_process = env.now
        

    data["wait_times"].append(time_start_process - time_at_queue)
    data["process_times"].append(time_end_process - time_start_process)

def setup(env, init, data):
    INIT_TASKS = init[0]
    N_MACHINES = init[1]

    MARKOV_TASK_DURATION = init[2]
    MARKOV_TASK_ARRIVAL = init[3]
    shortest_job_first = init[4]
    time_choice = init[5]

    server = Server(env, N_MACHINES)
    waiting_times = []

    for i in range(INIT_TASKS):
        env.process(process_task(env, server, data, waiting_times, MARKOV_TASK_DURATION, shortest_job_first, time_choice))

    while True:
        yield env.timeout(generate_random_markov(MARKOV_TASK_ARRIVAL))
        i += 1
        # if server.queue_length > 0:
        #     print(server.machine.queue)
        env.process(process_task(env, server, data, waiting_times, MARKOV_TASK_DURATION, shortest_job_first, time_choice))


def generate_random_markov(lamda):
    """
    Markov CDF: y = 1 - e^(-lamda t)
    The y value has a range of 0 to 1, which we can sample.
    Thus we can obtain a random t value by sampling y

    t = - ln(1 - y) / lamda
    """

    r = random.random()
    return -math.log(1 - r) / lamda


In [9]:
def run(init):
    data = {
    "wait_times": [],
    "process_times": [],
    }
    env = simpy.Environment()
    env.process(setup(env, init, data))
    env.run(until=MAX_SIM_TIME)
    
    
    print(f"Avg wait time: {np.mean(data['wait_times'])}")
    print(f"Avg process time: {np.mean(data['process_times'])}")

In [10]:
INIT_TASKS = 4
N_MACHINES = 1
MAX_SIM_TIME = 10000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 0.9
shortest_job_first = True
time_choice = 'm'

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL, shortest_job_first, time_choice]

print("Shortest job first:")
run(init)

shortest_job_first = False

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL, shortest_job_first, time_choice]

print("First in first out:")
run(init)

Shortest job first:
Avg wait time: 3.248558071967208
Avg process time: 1.009686086267107
First in first out:
Avg wait time: 14.945033598470092
Avg process time: 1.0078024090487203


In [11]:
INIT_TASKS = 4
N_MACHINES = 2
MAX_SIM_TIME = 10000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 1.8
shortest_job_first = True

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL, shortest_job_first, time_choice]

print("Shortest job first:")
run(init)

shortest_job_first = False

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL, shortest_job_first, time_choice]

print("First in first out:")
run(init)

Shortest job first:
Avg wait time: 1.6949542628520597
Avg process time: 0.9994817380601897
First in first out:
Avg wait time: 3.4744617921904406
Avg process time: 1.0039105594979054


In [16]:
INIT_TASKS = 4
N_MACHINES = 1
MAX_SIM_TIME = 100000

MARKOV_TASK_DURATION = 1
MARKOV_TASK_ARRIVAL = 0.9
shortest_job_first = True
time_choice = 'd'

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL, shortest_job_first, time_choice]

print("Shortest job first:")
run(init)

shortest_job_first = False

init = [INIT_TASKS, N_MACHINES, MARKOV_TASK_DURATION, MARKOV_TASK_ARRIVAL, shortest_job_first, time_choice]

print("First in first out:")
run(init)

Shortest job first:
Avg wait time: 4.179271022732335
Avg process time: 1.0
First in first out:
Avg wait time: 4.3543113202361745
Avg process time: 1.0
